In [1]:
import torch
import torch.nn as nn
from helchriss.utils import stprint

# Domain Definition
create function domains and evaluate using executor.

In [2]:
from domains.math.integer_domain import integer_executor
r_int = integer_executor.evaluate("plus(one(), two())", {})
stprint(r_int)

from domains.spatial.circle_domain import circle_executor
r_circ = circle_executor.evaluate("area(unit_circle())", {})
stprint(r_circ)

30 23:00:50 Function one automatically registered.
30 23:00:50 Function two automatically registered.
30 23:00:50 Function three automatically registered.
30 23:00:50 Function plus automatically registered.
30 23:00:50 Function smaller automatically registered.
30 23:00:50 Function bigger automatically registered.
30 23:00:50 Function N automatically registered.
30 23:00:51 Function Z automatically registered.
30 23:00:51 Function subset automatically registered.
30 23:00:51 Function unit_circle automatically registered.
30 23:00:51 Function area automatically registered.
30 23:00:51 Function disconnected automatically registered.
30 23:00:51 Function externally_connected automatically registered.
30 23:00:51 Function partial_overlap automatically registered.
30 23:00:51 Function equal automatically registered.
30 23:00:51 Function tangential_proper_part automatically registered.
30 23:00:51 Function non_tangential_proper_part automatically registered.
30 23:00:51 Function tangential_p

r_int = Value:[3.0]-int P:[1.0] (Value)
r_circ = Value:[tensor([3.1416])]-region P:[1.0] (Value)


# Rewrite Rules
   1. a `LocalFrame` module is several rewrite rules that shares the same arg writer.
   2. a NeuralRewriter module compose of a dictionary of LocalFrame.

In [3]:
from core.metaphors.rewrite import NeuralRewriter, LocalFrame
rewriter = NeuralRewriter()
#frame = LocalFrame()
#rewriter.add_frame("rw", frame)

# Rewrite Executor
**Property**
1) it stores a `ExecutorGroup` that can look up domain functions.
2) a rewriter of class `NeuralRewriter`, is stores all the rewrite rules learned.

**Methods**
1) `evaluate(expr, grounding)` method takes a expression and grounding as input, then calculate the exepected output of an expression over all rewrite rules. The output should be a single value as the exepected output.
2) `infer_rewrite` method takes an expression (and grounding) as input and propose several rewrite rules

In [ ]:
from core.metaphors.executor import ExecutorGroup, RewriteExecutor
from helchriss.knowledge.symbolic import Expression
executor = RewriteExecutor(ExecutorGroup([circle_executor, integer_executor]))
result = executor.evaluate("plus:Integer(one:Integer(), two:Integer())", {})
print(result)
executor.base_executor.register_function("idd", lambda x : x)

expr = "partial_overlap:Circle(N:Integer(), Z:Integer())"
#expr = "partial_overlap:Circle(N:Integer(), unit_circle:Circle())"
expr = Expression.parse_program_string(expr)
hypothesis = executor.infer_rewrite_expr(expr)
stprint(hypothesis)
executor.add_metaphors(hypothesis) ### add rewrite exprs

print(executor.base_executor._function_registry)

import graphviz
#dot_content = executor.rewriter._display_dot_graph(executor.rewriter.rewrite_rules())
#graph = graphviz.Source(dot_content)
#graph.render("namo", format='png', cleanup=True)
#graph.view()  # Opens the image

executor.evaluate(expr, grounding = {})

31 02:54:50 WRN The input domain is empty, creating an empty domain
31 02:54:50 WRN The input domain is empty, creating an empty domain


1
1
1
Value:[3.0]-int:Integer P:[1.0]
hypothesis = list (length: 1)[
  list (length: 3)[
    0: 'partial_overlap:Circle'
    list (length: 2)[
      0: state:Circle (TypeBase)
      1: state:Circle (TypeBase)
    ]
    list (length: 2)[
      0: set:Integer (TypeBase)
      1: set:Integer (TypeBase)
    ]
  ]
]
['subset:Integer']
list (length: 2)[
  0: set:Integer (TypeBase)
  1: set:Integer (TypeBase)
]


AttributeError: 'ExecutorGroup' object has no attribute '_function_registry'

In [ ]:
"""vector[float, [4, 5]]"""
from core.metaphors.types import fill_hole
from helchriss.dsl.dsl_types import TypeBase

in_type = [TypeBase("boolean"), TypeBase("vector[float,[2,3]]")]
out_type = TypeBase("vector[float,[5]]")

filler = fill_hole(in_type, out_type)

output = filler(torch.tensor(1.0), torch.tensor([[1.0,2.0,3.0],[1.0,1.0,-1.]]))
print(output)

Value:[tensor([ 0.2410,  0.0690, -0.3061, -0.0783,  0.0100], grad_fn=<ViewBackward0>)]-vector[float,[5]] P:[1.0]
